In [1]:
#1.-*- coding: utf-8 -*-
import os
import sys
import logging
import traceback

# 프로젝트 루트 경로 설정 (노트북 위치에 따라 조정 필요)
# 예: 노트북이 kospi_relation_rag_pipeline 폴더 내에 있다면
# project_root = os.path.abspath(os.path.join(os.getcwd()))
# 노트북이 kospi_relation_rag_pipeline 폴더 밖에 있다면 경로 직접 지정
project_root = os.path.abspath(os.path.join(os.getcwd())) # 실제 경로로 수정!
sys.path.append(project_root)


# core 모듈 및 config 임포트
# (만약 ModuleNotFoundError 발생 시 위 project_root 경로 확인)
try:
    from core import document_processor, vector_store_manager, llm_extractor, matrix_builder
    from core.index_setup import create_search_index # 인덱스 생성 함수 임포트
    from config import logger, KOSPI100_LIST_PATH, TARGET_YEAR, DATA_DIR
except ImportError as e:
    print(f"모듈 임포트 오류: {e}")
    print("프로젝트 경로 설정(project_root) 또는 가상환경 활성화 상태를 확인하세요.")
    # 필요한 경우 추가 디버깅 정보 제공

# PDF 파일 입력 디렉토리 정의
INPUT_PDF_DIR = os.path.join(DATA_DIR, 'input_pdfs')
# 출력 매트릭스 저장 디렉토리 정의 (없으면 생성)
OUTPUT_MATRIX_DIR = os.path.join(project_root, "output_matrices")
os.makedirs(OUTPUT_MATRIX_DIR, exist_ok=True)


logger.info("===== 노트북 셀 1: 초기 설정 및 임포트 완료 =====")
print(f"프로젝트 루트: {project_root}")
print(f"PDF 입력 디렉토리: {INPUT_PDF_DIR}")
print(f"KOSPI 100 리스트 경로: {KOSPI100_LIST_PATH}")

2025-04-18 15:03:43,726 - config - INFO - ===== 노트북 셀 1: 초기 설정 및 임포트 완료 =====


프로젝트 루트: /Users/nsj/Desktop/kospi_relation_rag_pipeline
PDF 입력 디렉토리: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs
KOSPI 100 리스트 경로: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/kospi100_stock_codes.txt


In [2]:
#2. Azure AI Search 인덱스 설정 확인 및 생성/업데이트 시도
# 이 셀은 전체 데이터 처리 전에 한 번만 실행하거나, 매번 실행하여 인덱스 상태를 보장할 수 있습니다.
logger.info("--- Azure AI Search 인덱스 설정 확인/시도 ---")
index_ready = create_search_index()

if index_ready:
    logger.info("--- 인덱스 준비 완료 ---")
else:
    logger.error("--- 인덱스 설정 실패. 이후 단계 진행 불가 ---")
    # 오류 발생 시 노트북 실행 중단 또는 사용자 확인 필요
    raise RuntimeError("Azure AI Search 인덱스 설정 실패") # 또는 pass

print(f"인덱스 준비 상태: {index_ready}")

2025-04-18 15:04:00,975 - config - INFO - --- Azure AI Search 인덱스 설정 확인/시도 ---
2025-04-18 15:04:00,977 - config - INFO - Azure AI Search 인덱스 'kospi100-rag-index' 생성 또는 업데이트 시도...
2025-04-18 15:04:00,980 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 15:04:00,980 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 15:04:00,981 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 15:04:00,981 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of cl

인덱스 준비 상태: True


In [3]:
# 3 KOSPI 100 종목 코드 목록 로드
stock_codes = []
if not os.path.exists(KOSPI100_LIST_PATH):
    logger.error(f"KOSPI 100 목록 파일을 찾을 수 없습니다: {KOSPI100_LIST_PATH}")
    # 여기서 오류를 발생시키거나 빈 리스트로 진행할 수 있습니다.
else:
    try:
        with open(KOSPI100_LIST_PATH, 'r', encoding='utf-8') as f:
            stock_codes = [line.strip() for line in f if line.strip() and not line.startswith('#')] # 주석 처리된 라인 제외
        logger.info(f"종목 코드 {len(stock_codes)}개 로드됨.")
    except Exception as e:
        logger.error(f"종목 코드 파일 로드 중 오류 발생: {e}")

# 테스트를 위해 일부 종목만 선택 (선택 사항)
# stock_codes = stock_codes[:5] # 예: 처음 5개 종목만 처리
print(f"처리 대상 종목 코드 (총 {len(stock_codes)}개): {stock_codes}")

2025-04-18 15:04:04,881 - config - INFO - 종목 코드 100개 로드됨.


처리 대상 종목 코드 (총 100개): ['005930', '000660', '105560', '005380', '012450', '068270', '035420', '000270', '055550', '005490', '207940', '012330', '086790', '373220', '035720', '259960', '316140', '000810', '138040', '030200', '006400', '009540', '028260', '033780', '051910', '034020', '010140', '017670', '329180', '064350', '015760', '402340', '066570', '323410', '000100', '032830', '042660', '009150', '096770', '267260', '003550', '011200', '047810', '003490', '352820', '018260', '034730', '010130', '021240', '086280', '024110', '005830', '180640', '079550', '090430', '003670', '267250', '032640', '028050', '051900', '071050', '000720', '326030', '042700', '161390', '016360', '010120', '006800', '271560', '047050', '241560', '036570', '034220', '009830', '010950', '035250', '004020', '011070', '011790', '097950', '128940', '011780', '005940', '078930', '066970', '011170', '022100', '036460', '251270', '450080', '029780', '302440', '454910', '383220', '005070', '377300', '018880', '361610

In [4]:
# 4.전체 기업 처리를 위한 루프 시작
# 노트북에서는 이 루프를 직접 실행하기보다,
# 아래 셀들을 특정 stock_code 하나에 대해 먼저 테스트하고,
# 전체 루프는 별도 스크립트(`.py`)에서 실행하거나 마지막에 노트북 셀로 실행하는 것이 좋습니다.

# --- 테스트를 위한 단일 stock_code 설정 ---
if stock_codes:
    test_stock_code = stock_codes[0] # 첫 번째 코드로 테스트
    logger.info(f"--- 단일 기업 테스트 시작: {test_stock_code} ---")
else:
    logger.error("처리할 종목 코드가 없습니다.")
    test_stock_code = None

# 이후 셀들에서는 test_stock_code 변수를 사용합니다.
# 전체 루프는 이 셀 아래의 로직을 for문으로 감싸면 됩니다.
# for stock_code in stock_codes:
#    test_stock_code = stock_code
#    (Cell 5 ~ Cell 11 로직)

2025-04-18 15:04:05,966 - config - INFO - --- 단일 기업 테스트 시작: 005930 ---


In [5]:
#5 현재 테스트 대상 stock_code
current_stock_code = test_stock_code
pdf_content_bytes = None # 초기화
pdf_file_path = None # 초기화

if current_stock_code:
    pdf_file_name = f"{current_stock_code}.pdf" # 파일 이름 형식 (필요시 수정)
    pdf_file_path = os.path.join(INPUT_PDF_DIR, pdf_file_name)
    logger.info(f"처리 대상 PDF 경로: {pdf_file_path}")

    if os.path.exists(pdf_file_path):
        try:
            with open(pdf_file_path, "rb") as f:
                pdf_content_bytes = f.read()
            logger.info(f"입력 PDF 파일 로드 성공: {pdf_file_path} ({len(pdf_content_bytes)} bytes)")
        except Exception as e:
            logger.error(f"PDF 파일 읽기 오류 ({current_stock_code}): {e}")
            # 오류 발생 시 pdf_content_bytes는 None 유지
    else:
        logger.warning(f"입력 PDF 파일을 찾을 수 없습니다: {pdf_file_path}")
        # 파일 없으면 pdf_content_bytes는 None 유지

else:
    logger.error("테스트할 stock_code가 설정되지 않았습니다.")

# 결과 확인
print(f"PDF 파일 경로: {pdf_file_path}")
# print(f"PDF 내용 (bytes): {pdf_content_bytes[:100]}...") # 앞부분 일부만 출력 (필요시)

2025-04-17 21:26:30,988 - config - INFO - 처리 대상 PDF 경로: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs/005930.pdf
2025-04-17 21:26:30,992 - config - INFO - 입력 PDF 파일 로드 성공: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs/005930.pdf (7263840 bytes)


PDF 파일 경로: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs/005930.pdf


In [6]:
#6 DI 분석 결과 초기화
di_result = None

# 이전 셀에서 PDF 내용이 성공적으로 로드되었을 경우에만 실행
if pdf_content_bytes:
    logger.info(f"--- {current_stock_code}: Document Intelligence 분석 시작 ---")
    try:
        # analyze_document_content 함수가 pdf_content_bytes (바이트)를 처리하는지 확인 필요
        # 만약 파일 경로를 받는다면 analyze_pdf_document(pdf_file_path) 호출
        di_result = document_processor.analyze_document_content(pdf_content_bytes)

        if di_result and di_result.content:
            logger.info(f"--- {current_stock_code}: Document Intelligence 분석 완료 ---")
            # print(f"Markdown 내용 미리보기:\n{di_result.content[:1000]}...") # 결과 Markdown 일부 출력
        elif di_result:
            logger.warning(f"--- {current_stock_code}: Document Intelligence 분석은 완료되었으나 content가 비어있습니다. ---")
        else:
            logger.warning(f"--- {current_stock_code}: Document Intelligence 분석 결과가 없습니다. ---")

    except Exception as e:
        logger.error(f"--- {current_stock_code}: Document Intelligence 분석 중 오류 발생: {e} ---")
        traceback.print_exc()
else:
    logger.warning(f"--- {current_stock_code}: PDF 내용이 없어 Document Intelligence 분석을 건너뜁니다. ---")

# 결과 확인
# print(di_result) # 전체 결과 객체 확인 (필요시)

2025-04-17 21:26:32,977 - config - INFO - --- 005930: Document Intelligence 분석 시작 ---
2025-04-17 21:26:32,981 - config - INFO - DI 클라이언트 초기화 성공
2025-04-17 21:26:32,982 - config - INFO - 바이트 데이터 분석 시작 (콘텐츠 길이: 7263840)
2025-04-17 21:26:32,983 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://documnetintelligence0417.cognitiveservices.azure.com//documentintelligence/documentModels/prebuilt-layout:analyze?api-version=REDACTED&outputContentFormat=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Length': '7263840'
    'content-type': 'application/octet-stream'
    'Accept': 'application/json'
    'x-ms-client-request-id': '327952c2-1b87-11f0-83cd-623dbcccadd6'
    'User-Agent': 'azsdk-python-ai-documentintelligence/1.0.2 Python/3.12.7 (macOS-10.16-x86_64-i386-64bit)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
A body is sent with the request
2025-04-17 21:26:46,564 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 202


In [7]:
#7. 섹션 및 전체 텍스트 초기화
sections = {}
full_text = ""

# 이전 셀에서 DI 결과가 정상적으로 나왔을 경우에만 실행
if di_result and di_result.content:
    logger.info(f"--- {current_stock_code}: 섹션 추출 시작 ---")
    try:
        # 섹션 필터링 없이 모든 H2/H3 섹션 추출 (이전 제안대로 함수 수정 가정)
        sections = document_processor.extract_sections_from_di_result(di_result)

        if sections:
            logger.info(f"추출된 섹션: {list(sections.keys())} (총 {len(sections)}개)")
            full_text = "\n\n".join(sections.values())
            logger.info(f"결합된 전체 텍스트 길이: {len(full_text)} 자")
        else:
            logger.warning(f"{current_stock_code}: DI 결과에서 유효한 섹션(H2/H3)을 추출하지 못했습니다. 전체 Markdown 내용을 사용합니다.")
            # 섹션 추출 실패 시, 전체 Markdown 내용을 full_text로 사용
            full_text = di_result.content

    except Exception as e:
        logger.error(f"--- {current_stock_code}: 섹션 추출 중 오류 발생: {e} ---")
        traceback.print_exc()
        logger.warning(f"{current_stock_code}: 섹션 추출 오류 발생. 전체 Markdown 내용을 사용합니다.")
        full_text = di_result.content # 오류 시 전체 내용 사용

else:
    logger.warning(f"--- {current_stock_code}: DI 결과가 없어 섹션 추출을 건너뜁니다. ---")

# 결과 확인
# print(f"추출된 섹션 내용 (첫번째 섹션 예시):\n{list(sections.values())[0][:500]}..." if sections else "섹션 없음")
# print(f"\n결합된 전체 텍스트 (앞부분):\n{full_text[:1000]}...")

2025-04-17 21:27:07,269 - config - INFO - --- 005930: 섹션 추출 시작 ---
2025-04-17 21:27:07,273 - config - INFO - Markdown H2/H3 기준 섹션 추출 완료: 총 4개
2025-04-17 21:27:07,274 - config - INFO - 추출된 섹션: ['대차잔고비중', 'General Information', '주주변동내역 [최근 20건]', '임직원 정보'] (총 4개)
2025-04-17 21:27:07,274 - config - INFO - 결합된 전체 텍스트 길이: 59835 자


In [8]:
#8.청크 초기화
text_chunks = []

# 이전 셀에서 full_text가 생성되었을 경우에만 실행
if full_text and full_text.strip():
    logger.info(f"--- {current_stock_code}: 텍스트 청킹 시작 ---")
    try:
        text_chunks = vector_store_manager.get_text_chunks(full_text)
        logger.info(f"텍스트 청킹 완료: 총 {len(text_chunks)}개 청크 생성")
    except Exception as e:
        logger.error(f"--- {current_stock_code}: 텍스트 청킹 중 오류 발생: {e} ---")
        traceback.print_exc()
else:
    logger.warning(f"--- {current_stock_code}: 청킹할 텍스트 내용이 없습니다. ---")

# 결과 확인
# print(f"생성된 청크 수: {len(text_chunks)}")
# print(f"첫번째 청크 미리보기:\n{text_chunks[0]}" if text_chunks else "청크 없음")

2025-04-17 21:27:09,589 - config - INFO - --- 005930: 텍스트 청킹 시작 ---
2025-04-17 21:27:09,597 - config - INFO - 텍스트 청크 분할 완료: 총 48개
2025-04-17 21:27:09,598 - config - INFO - 텍스트 청킹 완료: 총 48개 청크 생성


In [9]:
#9. 임베딩 결과 초기화
valid_chunks = []
embeddings = []

# 이전 셀에서 청크가 생성되었을 경우에만 실행
if text_chunks:
    logger.info(f"--- {current_stock_code}: 임베딩 생성 시작 (총 {len(text_chunks)}개 청크) ---")
    processed_count = 0
    for i, chunk in enumerate(text_chunks):
        # 너무 짧거나 공백만 있는 청크는 건너뛸 수 있음 (선택 사항)
        if not chunk or not chunk.strip():
            # logger.warning(f"비어있는 청크 건너뜀: index {i}")
            continue

        try:
            embedding = vector_store_manager.get_embedding(chunk)
            if embedding:
                embeddings.append(embedding)
                valid_chunks.append(chunk) # 임베딩 성공한 청크만 저장
                processed_count += 1
                if processed_count % 50 == 0: # 50개마다 로그 출력
                    logger.info(f"임베딩 진행 중... ({processed_count}/{len(text_chunks)})")
            else:
                # get_embedding 함수가 오류 시 빈 리스트나 None 반환 가정
                logger.warning(f"청크 {i}에 대한 임베딩 생성 실패 또는 빈 결과 반환. 건너뜁니다.")

        except Exception as e:
            logger.error(f"청크 {i} 임베딩 생성 중 오류 발생: {e}")
            # 특정 청크 오류 시 계속 진행할지 결정 (여기서는 건너뜀)
            # traceback.print_exc() # 필요시 상세 오류 출력

    logger.info(f"--- {current_stock_code}: 임베딩 생성 완료 ({len(embeddings)} / {len(text_chunks)} 성공) ---")
else:
    logger.warning(f"--- {current_stock_code}: 임베딩할 텍스트 청크가 없습니다. ---")

# 결과 확인
print(f"생성된 임베딩 수: {len(embeddings)}")
print(f"유효한 청크 수: {len(valid_chunks)}")
# print(f"첫번째 임베딩 벡터 (앞부분): {embeddings[0][:10]}..." if embeddings else "임베딩 없음")

2025-04-17 21:27:12,091 - config - INFO - --- 005930: 임베딩 생성 시작 (총 48개 청크) ---
2025-04-17 21:27:12,146 - config - INFO - Azure OpenAI 클라이언트 초기화 성공
2025-04-17 21:27:12,999 - httpx - INFO - HTTP Request: POST https://gptapi0417.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-04-17 21:27:13,037 - config - INFO - Azure OpenAI 클라이언트 초기화 성공
2025-04-17 21:27:13,869 - httpx - INFO - HTTP Request: POST https://gptapi0417.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-04-17 21:27:13,899 - config - INFO - Azure OpenAI 클라이언트 초기화 성공
2025-04-17 21:27:14,721 - httpx - INFO - HTTP Request: POST https://gptapi0417.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-04-17 21:27:14,764 - config - INFO - Azure OpenAI 클라이언트 초기화 성공
2025-04-17 21:27:15,588 - httpx - INFO - HTTP Request: POST https://gptapi0417.

생성된 임베딩 수: 48
유효한 청크 수: 48


In [10]:
#10.업로드 결과 초기화
upload_success = False

# 이전 셀에서 유효한 임베딩이 생성되었을 경우에만 실행
if valid_chunks and embeddings and len(valid_chunks) == len(embeddings):
    logger.info(f"--- {current_stock_code}: Azure AI Search 업로드 시작 ({len(valid_chunks)}개 문서) ---")
    try:
        # source_document_id로 pdf 파일 경로 또는 stock_code 전달
        upload_success = vector_store_manager.upsert_documents_to_ai_search(
            text_chunks=valid_chunks,
            embeddings=embeddings,
            source_document_id=pdf_file_path # 또는 current_stock_code 사용
        )
        if upload_success:
            logger.info(f"--- {current_stock_code}: Azure AI Search 업로드 완료 ---")
        else:
            logger.warning(f"--- {current_stock_code}: Azure AI Search 업로드 중 일부 또는 전체 실패 (로그 확인 필요) ---")

    except Exception as e:
        logger.error(f"--- {current_stock_code}: Azure AI Search 업로드 중 예외 발생: {e} ---")
        traceback.print_exc()
else:
    logger.warning(f"--- {current_stock_code}: 업로드할 유효한 청크/임베딩 데이터가 없습니다. ---")

# 결과 확인
print(f"AI Search 업로드 성공 여부: {upload_success}")

2025-04-17 21:28:04,216 - config - INFO - --- 005930: Azure AI Search 업로드 시작 (48개 문서) ---
2025-04-17 21:28:04,218 - config - INFO - Azure AI Search 클라이언트 초기화 성공 (인덱스: kospi100-rag-index)
2025-04-17 21:28:04,219 - config - INFO - Azure AI Search 업로드 준비: 총 48개 문서
2025-04-17 21:28:04,264 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://aiserach0417.search.windows.net/indexes('kospi100-rag-index')/docs/search.index?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1757560'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '68e1a83c-1b87-11f0-83cd-623dbcccadd6'
    'User-Agent': 'azsdk-python-search-documents/11.5.2 Python/3.12.7 (macOS-10.16-x86_64-i386-64bit)'
A body is sent with the request
2025-04-17 21:28:06,816 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 200
Response headers:
    'Transfer

AI Search 업로드 성공 여부: True


In [ ]:
# 11. --- 이 부분은 현재 루프 구조와 맞지 않으므로 재설계 필요 ---
# LLM 추출과 Matrix 생성은 모든 문서 처리가 끝난 후에,
# Azure AI Search에 저장된 데이터를 기반으로 수행하는 것이 일반적입니다.
# 예를 들어, 특정 기업 쌍에 대해 관련된 청크들을 AI Search에서 검색한 후,
# 그 청크들을 컨텍스트로 LLM에 질문하는 방식 등을 고려해야 합니다.

# 아래 코드는 마지막 처리된 기업의 데이터로만 동작하므로 주석 처리 또는 제거 권장

# logger.info(f"--- {current_stock_code}: LLM 관계 추출 및 Matrix 생성 시작 (예시) ---")
# if 'text_chunks' in locals() and text_chunks: # 변수가 정의되어 있고 내용이 있을 때만 실행
#     try:
#         # 예시: 마지막 처리된 기업의 text_chunks 사용 (논리적으로 맞지 않음)
#         prompt = llm_extractor.create_relationship_extraction_prompt(
#             context_chunks=text_chunks[:5], # 예시로 일부 청크만 사용
#             company_a_name="CompanyA", # 실제 회사 이름 필요
#             company_b_name="CompanyB"  # 실제 회사 이름 필요
#         )
#         # llm_response = llm_extractor.extract_relationship_with_llm(prompt) # 실제 LLM 호출 구현 필요
#         # relationships = llm_extractor.parse_llm_json_output(llm_response) # 실제 파싱 구현 필요
#         relationships = {} # 임시 값

#         matrix = matrix_builder.build_adjacency_matrix(relationships) # 실제 관계 데이터 필요
#         matrix_file_path = os.path.join(OUTPUT_MATRIX_DIR, f"{current_stock_code}_adjacency_matrix.npz")
#         matrix_builder.save_matrix(matrix, matrix_file_path)
#         logger.info(f"--- {current_stock_code}: Matrix 생성 완료 ---")
#     except Exception as e:
#         logger.error(f"--- {current_stock_code}: LLM 추출 또는 Matrix 생성 중 오류: {e} ---")
#         traceback.print_exc()
# else:
#     logger.warning(f"--- {current_stock_code}: LLM 추출/Matrix 생성 위한 데이터 부족 ---")